In [1]:
import torch
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
iris = sns.load_dataset('iris')

In [3]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [10]:
X_iris = iris.drop('species', axis=1)
X_iris

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [12]:
y_iris = iris['species']
y_iris

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object

In [119]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, train_size=0.7)

In [120]:
y_test = y_test.replace({'virginica' : 2, 'setosa' : 0, 'versicolor' : 1})

# setosa = 1
# versicolor = 2
# virginica = 3

In [121]:
y_train = y_train.replace({'virginica' : 2, 'setosa' : 0, 'versicolor' : 1})

# setosa = 1
# versicolor = 2
# virginica = 3

In [122]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
108,6.7,2.5,5.8,1.8
66,5.6,3.0,4.5,1.5
130,7.4,2.8,6.1,1.9
76,6.8,2.8,4.8,1.4
135,7.7,3.0,6.1,2.3
...,...,...,...,...
46,5.1,3.8,1.6,0.2
68,6.2,2.2,4.5,1.5
81,5.5,2.4,3.7,1.0
7,5.0,3.4,1.5,0.2


In [123]:
X_test.shape

(45, 4)

In [124]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.7)

In [125]:
X_test.shape

(31, 4)

In [126]:
X_val.shape

(14, 4)

In [127]:
y_val.shape, y_test.shape

((14,), (31,))

In [128]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()


In [129]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [130]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [131]:
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)

In [132]:
y_train.shape, y_test.shape, X_test.shape, X_train.shape

(torch.Size([105]),
 torch.Size([31]),
 torch.Size([31, 4]),
 torch.Size([105, 4]))

In [133]:

class IrisNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(IrisNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_hidden_neurons, 24)
        self.act1 = torch.nn.ReLU()
        
        self.fc2 = torch.nn.Linear(24, 24)
        self.act2 = torch.nn.ReLU()
        
        self.fc3 = torch.nn.Linear(24, 3)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        
        x = self.fc2(x)
        x = self.act2(x)
        
        x = self.fc3(x)
        return x
    
iris_net = IrisNet(4)
        
        

In [134]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(iris_net.parameters(), lr=1.0e-3)

In [135]:
len(X_train), len(y_train)

(105, 105)

In [143]:
def saveModel():
    path = r'/home/alex/jupyter/NetModel.pth'
    torch.save(iris_net.state_dict(), path)
    

In [158]:
# обучение будем проводить по батчам

batch_size = 4


def train(num_epoch):
    best_accuracy = 0.
    print('Тренировка началась...')
    
    for epoch in range(1, num_epoch+1):
        running_train_loss = 0.0
        running_accuracy = 0.0
        running_val_loss = 0.0
        total = 0.0
        
        order = np.random.permutation(len(X_train))
        
        for start_index in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
        
            batch_indexes = order[start_index : start_index + batch_size]
        
            x_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]
        
            preds = iris_net.forward(x_batch)
        
            train_loss = loss(preds, y_batch)
            train_loss.backward()
        
            optimizer.step()
            
            running_train_loss += train_loss.item()
            
        train_loss_value = running_train_loss / len(X_train)
        
        # Валидация
        with torch.no_grad():
            iris_net.eval()
            order_2 = np.random.permutation(len(X_val))
            
            for start_index_2 in range(0, len(X_val), batch_size):
        
                batch_indexes_2 = order_2[start_index_2 : start_index_2 + batch_size]
        
                x_batch = X_val[batch_indexes_2]
                y_batch = y_val[batch_indexes_2]
        
                preds_2 = iris_net.forward(x_batch)
        
                val_loss = loss(preds_2, y_batch)
                
                
                _, predicted = torch.max(preds_2, 1)
                running_val_loss += val_loss.item()
                total += y_batch.size(0)
                running_accuracy += (predicted == y_batch).sum().item()
                
        val_loss_value = running_val_loss / len(X_test)
        
        accuracy = (100 * running_accuracy / total)
        
        if accuracy > best_accuracy: 
            saveModel() 
            best_accuracy = accuracy 
        
        print('Тренировка на батче', epoch, 'Тренировочный Loss: %.4f' %train_loss_value, 'Валидационный Loss: %.4f' %val_loss_value, 'Accuracy is %d %%' % (accuracy))

                
        
train(10)

Тренировка началась...
Тренировка на батче 1 Тренировочный Loss: 0.0298 Валидационный Loss: 0.0063 Accuracy is 100 %
Тренировка на батче 2 Тренировочный Loss: 0.0333 Валидационный Loss: 0.0061 Accuracy is 100 %
Тренировка на батче 3 Тренировочный Loss: 0.0311 Валидационный Loss: 0.0065 Accuracy is 100 %
Тренировка на батче 4 Тренировочный Loss: 0.0306 Валидационный Loss: 0.0059 Accuracy is 100 %
Тренировка на батче 5 Тренировочный Loss: 0.0285 Валидационный Loss: 0.0049 Accuracy is 100 %
Тренировка на батче 6 Тренировочный Loss: 0.0274 Валидационный Loss: 0.0046 Accuracy is 100 %
Тренировка на батче 7 Тренировочный Loss: 0.0304 Валидационный Loss: 0.0050 Accuracy is 100 %
Тренировка на батче 8 Тренировочный Loss: 0.0329 Валидационный Loss: 0.0043 Accuracy is 100 %
Тренировка на батче 9 Тренировочный Loss: 0.0255 Валидационный Loss: 0.0041 Accuracy is 100 %
Тренировка на батче 10 Тренировочный Loss: 0.0254 Валидационный Loss: 0.0047 Accuracy is 100 %


In [138]:
!pwd

/home/alex/jupyter


In [153]:
def test():
    running_accuracy = 0.
    total = 0
    
    with torch.no_grad():
        model = IrisNet(4)
        path = '/home/alex/jupyter/NetModel.pth'
        model.load_state_dict(torch.load(path))
        for _ in range(len(X_test)):
            preds = model(X_test)
            _, predicted = torch.max(preds, 1)
            total += y_test.size(0)
            running_accuracy += (predicted == y_test).sum().item()
            
        print('Точность модели на тестовой выборке', ': %d %%' % (100 * running_accuracy / total))    

    

In [154]:
test()

Точность модели на тестовой выборке : 100 %


In [157]:
iris_net.parameters

<bound method Module.parameters of IrisNet(
  (fc1): Linear(in_features=4, out_features=24, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=24, out_features=24, bias=True)
  (act2): ReLU()
  (fc3): Linear(in_features=24, out_features=3, bias=True)
)>